In [4]:
import pandas as pd
import numpy as np
from scipy import stats
import pyreadstat

In [5]:
#%pip install pyreadstat

In [ ]:
# .sav 파일 로드 (Pyreadstat를 사용하여 .sav 파일 불러오기)
df_sav, meta = pyreadstat.read_sav("한국행정연구원_공직생활실태조사_데이터_2023.SAV")

In [7]:
# 변수 생성 (설문 항목 평균 계산)
df_sav['transformational_leadership'] = df_sav[['q20_3', 'q20_4', 'q20_5', 'q20_6']].mean(axis=1)
df_sav['transactional_leadership'] = df_sav[['q20_1', 'q20_2']].mean(axis=1)
df_sav['goal_clarity'] = df_sav[['q19_1', 'q19_2', 'q19_3', 'q19_4']].mean(axis=1)
df_sav['role_ambiguity'] = df_sav[['q31_4', 'q31_5', 'q31_6']].mean(axis=1)
df_sav['organizational_performance'] = df_sav[['q25_1', 'q25_2']].mean(axis=1)

In [8]:
df_sav

,ID,q1,q1_1_m1,q1_1_m2,q2,q3_1,q3_2,q3_3,q4_1,q4_2,q4_3,q5_1,q5_2,q5_3,q5_1_1,q5_2_1,q5_3_1,q6_1,q6_2,q6_3,q6_4,q6_1_1,q6_2_1,q6_3_1,q6_4_1,a1,a2,q7_1,...,dq2,dq3,dq4_1,dq4_2,dq5_1,dq5_2,dq6,dq7,dq8,dq9_1,dq9_2,dq10,dq11,orgtype,DM1,DM2,DM3,DM4,DM5,DM6,DM7,WT,TWT,transformational_leadership,transactional_leadership,goal_clarity,role_ambiguity,organizational_performance
0,1.0,3.0,NaN,NaN,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,3.0,...,1976.0,1.0,2.0,2.0,3.0,3.0,1.0,4.0,1.0,9.0,7.0,2.0,4.0,6.0,3.0,1.0,3.0,3.0,4.0,1.0,3.0,52.275000,1.559088,4.00,4.0,4.00,4.000000,4.0
1,2.0,3.0,NaN,NaN,4.0,4.0,4.0,4.0,4.0,5.0,4.0,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,3.0,1.0,4.0,...,1981.0,2.0,0.0,0.0,3.0,3.0,2.0,3.0,1.0,9.0,6.0,2.0,4.0,5.0,2.0,2.0,3.0,3.0,3.0,1.0,3.0,23.871429,0.712258,4.00,4.0,4.00,2.000000,4.0
2,3.0,3.0,NaN,NaN,2.0,3.0,4.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,5.0,2.0,4.0,...,1996.0,2.0,0.0,0.0,3.0,3.0,3.0,1.0,1.0,9.0,8.0,1.0,5.0,6.0,3.0,2.0,1.0,3.0,1.0,1.0,4.0,47.556250,1.418352,3.00,2.5,2.75,3.333333,3.5
3,4.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,2.0,2.0,1.0,NaN,NaN,4.0,4.0,1.0,2.0,1.0,4.0,NaN,4.0,NaN,1.0,1.0,4.0,...,1996.0,2.0,0.0,0.0,3.0,3.0,2.0,1.0,1.0,9.0,9.0,1.0,5.0,6.0,3.0,2.0,1.0,3.0,1.0,1.0,4.0,58.010000,1.730132,4.25,4.0,4.00,2.000000,4.5
4,5.0,5.0,3.0,2.0,4.0,2.0,4.0,4.0,3.0,2.0,3.0,2.0,2.0,1.0,NaN,NaN,4.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,1990.0,2.0,0.0,0.0,3.0,3.0,3.0,1.0,1.0,7.0,6.0,2.0,3.0,2.0,1.0,2.0,2.0,3.0,1.0,1.0,3.0,2.666667,0.079520,3.00,3.0,4.00,2.000000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6439,6454.0,4.0,3.0,5.0,3.0,2.0,3.0,3.0,4.0,4.0,4.0,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,5.0,1.0,3.0,...,1971.0,1.0,3.0,0.0,3.0,4.0,1.0,6.0,1.0,9.0,6.0,2.0,4.0,6.0,3.0,2.0,4.0,4.0,6.0,1.0,3.0,44.758824,1.334920,3.00,3.0,3.00,2.000000,4.0
6440,6455.0,3.0,NaN,NaN,4.0,3.0,3.0,4.0,4.0,4.0,4.0,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,3.0,...,1974.0,1.0,0.0,0.0,3.0,3.0,2.0,4.0,2.0,9.0,6.0,2.0,4.0,6.0,3.0,1.0,4.0,3.0,4.0,2.0,3.0,44.758824,1.334920,3.75,3.0,3.50,3.000000,3.5
6441,6456.0,4.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1.0,3.0,1.0,NaN,NaN,3.0,NaN,2.0,2.0,3.0,...,1969.0,1.0,2.0,0.0,2.0,2.0,2.0,6.0,1.0,9.0,6.0,2.0,3.0,6.0,3.0,1.0,4.0,2.0,6.0,1.0,3.0,56.595122,1.687934,3.00,3.0,3.25,2.000000,3.5
6442,6457.0,4.0,3.0,6.0,3.0,2.0,4.0,4.0,4.0,4.0,4.0,2.0,2.0,2.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,3.0,1.0,4.0,...,1967.0,1.0,2.0,0.0,3.0,3.0,1.0,6.0,1.0,5.0,5.0,3.0,3.0,5.0,2.0,1.0,4.0,3.0,6.0,1.0,2.0,19.400000,0.578843,5.00,5.0,5.00,1.000000,4.0


In [9]:
# 상관 분석에 사용할 데이터 추출 (통제변수 포함)
df_selected = df_sav[['transformational_leadership', 'transactional_leadership', 'goal_clarity', 'role_ambiguity', 
                      'organizational_performance', 'DM2', 'DM3', 'dq3', 'DM4', 'DM5', 'DM7', 'DM6', 'DM1']]


In [ ]:
# 상관 계수 계산
correlation_matrix = df_selected.corr()

In [10]:
# p-value 계산 함수

def calculate_p_values(correlation_matrix, n):
    p_values = pd.DataFrame(np.ones(correlation_matrix.shape), columns=correlation_matrix.columns, index=correlation_matrix.index)
    for i in range(correlation_matrix.shape[0]):
        for j in range(i + 1, correlation_matrix.shape[1]): # 상삼각 행렬만 계산
            corr = correlation_matrix.iloc[i, j]
            t_stat = corr * np.sqrt((n - 2) / (1 - corr**2)) # t-통계량 계산
            p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=n - 2)) # p-value 계산
            p_values.iloc[i, j] = p_values.iloc[j, i] = p_value
    return p_values


# p-value 계산

n = len(df_sav)
p_values = calculate_p_values(correlation_matrix, n)


# 유의수준에 따른 별표 추가

def apply_significance_levels(correlation_matrix, p_values):
    significance_matrix = correlation_matrix.copy()
    for i in range(correlation_matrix.shape[0]):
        for j in range(i + 1, correlation_matrix.shape[1]):
            p_value = p_values.iloc[i, j]
            if p_value < 0.001:
                significance_matrix.iloc[i, j] = f"{correlation_matrix.iloc[i, j]:.2f} ***"
            elif p_value < 0.01:
                significance_matrix.iloc[i, j] = f"{correlation_matrix.iloc[i, j]:.2f} **"
            elif p_value < 0.05:
                significance_matrix.iloc[i, j] = f"{correlation_matrix.iloc[i, j]:.2f} *"
            else:
                significance_matrix.iloc[i, j] = f"{correlation_matrix.iloc[i, j]:.2f}"
    return significance_matrix


# 유의수준을 적용한 상관 행렬
final_significance_matrix = apply_significance_levels(correlation_matrix, p_values)

<positron-console-cell-10>:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.85 ***' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
<positron-console-cell-10>:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.55 ***' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
<positron-console-cell-10>:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.38 ***' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
<positron-console-cell-10>:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.52 ***' has dtype incompatible with float64, please explicit

In [13]:
final_significance_matrix

,transformational_leadership,transactional_leadership,goal_clarity,role_ambiguity,organizational_performance,DM2,DM3,dq3,DM4,DM5,DM7,DM6,DM1
transformational_leadership,1.000000,0.85 ***,0.55 ***,-0.38 ***,0.52 ***,-0.07 ***,0.05 ***,-0.04 **,-0.02,0.08 ***,-0.04 ***,0.04 **,-0.02
transactional_leadership,0.852289,1.0,0.54 ***,-0.35 ***,0.48 ***,-0.08 ***,0.08 ***,-0.07 ***,-0.01,0.11 ***,-0.06 ***,0.04 **,-0.03 *
goal_clarity,0.545583,0.537447,1.0,-0.36 ***,0.62 ***,-0.07 ***,0.17 ***,-0.11 ***,0.03 *,0.19 ***,-0.10 ***,0.11 ***,-0.03 **
role_ambiguity,-0.381259,-0.348962,-0.363586,1.0,-0.35 ***,0.05 ***,-0.13 ***,0.08 ***,-0.01,-0.15 ***,0.11 ***,-0.05 ***,0.05 ***
organizational_performance,0.518545,0.48277,0.622609,-0.351004,1.0,-0.05 ***,0.13 ***,-0.08 ***,0.01,0.16 ***,-0.08 ***,0.08 ***,-0.00
DM2,-0.071976,-0.081197,-0.073634,0.049955,-0.052674,1.0,-0.16 ***,0.14 ***,0.01,-0.08 ***,0.14 ***,-0.03 **,0.10 ***
DM3,0.054264,0.084862,0.17002,-0.130319,0.130107,-0.160688,1.0,-0.57 ***,0.17 ***,0.84 ***,-0.54 ***,0.10 ***,-0.08 ***
dq3,-0.040776,-0.068662,-0.106773,0.076897,-0.077565,0.135936,-0.565838,1.0,-0.13 ***,-0.55 ***,0.41 ***,-0.08 ***,0.07 ***
DM4,-0.018689,-0.013845,0.029269,-0.005802,0.010518,0.011372,0.172931,-0.126956,1.0,0.08 ***,-0.26 ***,0.11 ***,-0.20 ***
DM5,0.079580,0.109901,0.189741,-0.151024,0.161575,-0.079243,0.838884,-0.551932,0.080935,1.0,-0.57 ***,0.02 *,-0.07 ***


In [14]:
final_significance_matrix.iloc[:5,:5]

,transformational_leadership,transactional_leadership,goal_clarity,role_ambiguity,organizational_performance
transformational_leadership,1.000000,0.85 ***,0.55 ***,-0.38 ***,0.52 ***
transactional_leadership,0.852289,1.0,0.54 ***,-0.35 ***,0.48 ***
goal_clarity,0.545583,0.537447,1.0,-0.36 ***,0.62 ***
role_ambiguity,-0.381259,-0.348962,-0.363586,1.0,-0.35 ***
organizational_performance,0.518545,0.48277,0.622609,-0.351004,1.0
